In [2]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './test_data'

논문을 벡터 db에 넣기

In [3]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

### 프롬프트

모델 선언

## TEST1

In [10]:
# LLM model
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,PromptTemplate



prompt =  ChatPromptTemplate(input_variables=['context', 'question'], 
    messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(
    input_variables=['context', 'question'], 
    template=
        '''You are an assistant for question-answering tasks. 
            Use the following pieces of retrieved context to answer the question. 
            If you don't know the answer, just say that you don't know. 
            Use three sentences maximum and keep the answer concise.
            \nQuestion: {question} 
            \nContext: {context} 
            \nAnswer:'''))
    ])
# 출처: https://rfriend.tistory.com/832 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]
model = ChatOpenAI(model_name="gpt-4o", temperature=0)


qa_chain = RetrievalQA.from_chain_type(
    llm=model, 
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

question  ="What is a LangChain?"
result = qa_chain.invoke({"query": question})

result["result"]
#출처: https://rfriend.tistory.com/832 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]

"I don't know. The provided context does not contain information about LangChain."

## TEST2

In [16]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt ="""채팅 기록과 최신 사용자 질문을 고려하여, 채팅 기록의 문맥을 참조할 수 있는 독립적인 질문을 작성하세요. 
질문에 답변하지 말고, 필요하다면 질문을 다시 작성한 후 그렇지 않으면 그대로 반환하세요."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])
chat_history.extend([HumanMessage(question), ai_msg_2["answer"]])

Task decomposition can be done in several ways, such as using techniques like Chain of Thought (CoT) or Tree of Thoughts to break down complex tasks into smaller steps. Common methods include providing simple prompts to language models (LLM) like "Steps for XYZ" or task-specific instructions such as "Write a story outline." Human inputs can also be used for task decomposition, allowing for a more customized and detailed breakdown of tasks.


In [20]:
print(type(chat_history[0]))

<class 'langchain_core.messages.human.HumanMessage'>


In [26]:
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model="gpt-4o", temperature=0)
## Define the prompt for the QA system
template = '''
너는 사회복지사의 업무를 도와주기 위한 챗봇이다. \\
사회복지 업무와 관련된 메뉴얼과 가이드북을 읽어서 사용자의 질문에 답변할 수 있도록 학습되었다. \\
너는 주어진 업무를 아주 잘 한다. \\
Answer the question based only on the following context:
{context}
'''
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


contextualize_q_system_prompt ="""채팅 기록과 최신 사용자 질문을 고려하여, 채팅 기록의 문맥을 참조할 수 있는 독립적인 질문을 작성하세요. 
질문에 답변하지 말고, 필요하다면 질문을 다시 작성한 후 그렇지 않으면 그대로 반환하세요."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


sys_prompt = '''
너는 사회복지사의 업무를 도와주기 위한 챗봇이다. \\
사회복지 업무와 관련된 메뉴얼과 가이드북을 읽어서 사용자의 질문에 답변할 수 있도록 학습되었다. \\
너는 주어진 업무를 아주 잘 한다. \\
Answer the question based only on the following context:
{context}
'''
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sys_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [29]:
chat = [{'role': 'user', 'message': '안녕'}, {'role': 'ai', 'message': '안녕하세요! 무엇을 도와드릴까요? 사회복지사 업무와 관련된 질문이 있으시면 말씀해 주세요.'}, {'role': 'user', 'message': '반가워'}, {'role': 'ai', 'message': '안녕하세요! 무엇을 도와드릴까요? 사회복지사 보수교육비 청구나 처우개선 지원사업에 대해 궁금한 점이 있으시면 말씀해 주세요.'}, {'role': 'user', 'message': '안녕'}, {'role': 'ai', 'message': '안녕하세요! 무엇을 도와드릴까요? 사회복지사 업무와 관련된 질문이 있으시면 말씀해 주세요.'}, {'role': 'user', 'message': '이전대화'}, {'role': 'ai', 'message': '이전 대화 내용이 제공되지 않았습니다. 질문을 다시 한 번 명확하게 해주시면, 제공된 맥락을 바탕으로 답변을 드리겠습니다. 어떤 정보가 필요하신가요?'}]
chat[-2:]

[{'role': 'user', 'message': '이전대화'},
 {'role': 'ai',
  'message': '이전 대화 내용이 제공되지 않았습니다. 질문을 다시 한 번 명확하게 해주시면, 제공된 맥락을 바탕으로 답변을 드리겠습니다. 어떤 정보가 필요하신가요?'}]

In [27]:

conversational_rag_chain.invoke(
    {"input": "안녕"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 5e01da21-d424-43ab-83ec-7427aaadf635 not found for run 1dee41ac-57c8-4fc6-a884-ce81a8de667b. Treating as a root run.


'안녕하세요! 무엇을 도와드릴까요? 사회복지 업무와 관련된 질문이 있으시면 언제든지 말씀해 주세요.'

In [24]:
from langchain_core.messages import AIMessage

for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: 내가 이전에 뭐라고 질문했지?

AI: 이전에 어떤 질문을 하셨는지에 대한 정보는 제공되지 않았습니다. 현재 제공된 정보만으로는 이전 질문을 알 수 없습니다. 새로운 질문이 있으시면 말씀해 주세요.

User: 안녕 내가 이전에 뭐라고 질문했어?

AI: 죄송하지만, 현재 시스템에서는 이전 대화 내용을 기억하거나 저장하지 않습니다. 새로운 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.



## ORIGINAL

In [3]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = '''
너는 사회복지사의 업무를 도와주기 위한 챗봇이다. \\
사회복지 업무와 관련된 메뉴얼과 가이드북을 읽어서 사용자의 질문에 답변할 수 있도록 학습되었다. \\
너는 주어진 업무를 아주 잘 한다. \\
Answer the question based only on the following context:
{context}

Question: {question}

'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)
# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
    사회복지시설에서의 업무를 수행하다가 발생한 사고들에 대해 어떻게 대처해야 할까요?
"""    # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> 사회복지시설에서의 업무를 수행하다가 발생한 사고들에 대해 다음과 같이 대처해야 합니다:
> 
> 1. **즉각적인 안전조치**:
>    - 사고 발생 시 즉각적인 안전조치를 취합니다. 예를 들어, 부상자가 발생한 경우 응급처치를 실시하고, 필요 시 응급의료 및 소방서에 연락합니다.
> 
> 2. **초기 대응 및 긴급 조치**:
>    - 폭력 사건이 발생하면 폭력피해 예방·관리 담당자가 주도하여 초기 대응을 합니다. 긴급한 조치가 이루어졌고 초기대응을 마친 후에는 사고 수습을 시행합니다.
> 
> 3. **침착한 대응**:
>    - 위급한 상황일 때 빠른 판단이 중요하지만, 침착한 대응이 제2의 피해를 최소화할 수 있으므로 침착하게 진행합니다.
> 
> 4. **사고 보고서 작성**:
>    - 사고보고서는 작성요령에 따라 사고 당사자가 문서화하는 것이 중요합니다.
> 
> 5. **사고 원인 및 대책 공유**:
>    - 동료 사회복지시설 종사자와 사고의 요인과 대책을 공유하여 재발하지 않도록 합니다. 사고 당사자에 대해서도 동료들이 심리적 지지를 해주는 것이 중요합니다.
> 
> 6. **의료비 보장**:
>    - 사회복지시설 종사자가 폭력으로 인해 의료비가 발생한 경우, 사회복지공제회의 ‘정부지원 단체 상해 공제’를 통해 보장받을 수 있도록 합니다.
> 
> 7. **정기적 점검 및 예방훈련**:
>    - 시설 내외 안전에 관한 정기적 점검을 실시하고, 사고 재발 및 폭력피해 예방을 위한 교육훈련 계획을 수립하여 지속적으로 훈련을 수행합니다.
> 
> 8. **법적 조치 및 대응**:
>    - 폭력 발생과 관련한 법적 조치, 대응, 보상 절차를 마련합니다.
> 
> 9. **직원 재배치 및 업무 변경**:
>    - 폭력 사고 관련 직원 재배치 및 업무 변경에 대한 조정을 실시합니다.
> 
> 10. **중재 및 해결 기술**:
>     - 가해 이용자 및 보호자와 피해자 간의 중재, 해결 기술을 적용합니다.
> 
> 11. **지속적인 모니터링**:
>     - 가해 이용자 및 피해 사회복지시설 종사자에 대한 지속적인 모니터링을 실시합니다.
> 
> 12. **대피 시 행동요령**:
>     - 대피 시 엘리베이터 사용을 피하고, 부상자 발생 시 응급처치를 실시하며, 대피 후 인원 확인 및 점검을 실시합니다. 2차 재난 발생에 따른 예방조치도 사전에 실시합니다.
> 
> 이와 같은 절차를 통해 사회복지시설에서 발생한 사고에 대해 체계적이고 신속하게 대처할 수 있습니다.

In [ ]:
class chaingpt:
    def __init__(self,api_key,retriever, sys_prompt="",model="gpt-4o"):
        self.template = sys_prompt + '''Answer the question based only on the following context:
        {context}

        Question: {question}
        '''
        self.prompt = ChatPromptTemplate.from_template(self.template)
        self.model = ChatOpenAI(api_key=api_key,model=model, temperature=1)
        self.chainmodel = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
        | self.prompt  # 프롬프트 템플릿을 적용합니다.
        | self.model  # 모델을 호출합니다.
        | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
        )
    def invoke(self,input_message):
        return self.chainmodel.invoke(input_message)
    
#ex
api_key = OPENAI_API_KEY
retriever = vectordb.as_retriever()
sys_prompt = """사용자의 외로움을 판단하고, 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. """
gpt = chaingpt(api_key,retriever,sys_prompt)
input_message =  """사용자의 외로움은 뭔가요? 적절한 대화상대가 되어주세요."""
print(gpt.invoke(input_message))
        